# Example Notebook to show how to predict solar power generation

Make sure to have installed the quart_solar_forecast package: 
```
pip install quartz_solar_forecast
```

In [ ]:
# Import forecast script and PVSite class.
from quartz_solar_forecast.forecast import run_forecast
from quartz_solar_forecast.pydantic_models import PVSite

# Import matplotlib and plotly for plotting.
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
# Create a PVSite object with the site's latitude, longitude and capacity.
site = PVSite(latitude=51.75, longitude=-1.25, capacity_kwp=1.25)

In [ ]:
# Run the forecast for a specific initial timestamp.
# This generates a forecast at 15 minute intervals for the following 48 hours.
predictions_df = run_forecast(site=site, ts='2023-11-01')
print(predictions_df)

In [ ]:
# Create an interactive plot of the forecast using plotly.
fig = px.line(predictions_df)
fig.show()

In [ ]:
# Create a static plot of the forecast using matplotlib.
predictions_df.plot()
plt.show()